In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset
# from Models.SeEANet import SeEANet
from Models.MyoNet import MyoNet
import torch.optim as optim
import torch.nn as nn
from datetime import datetime
from utils.tools import make_datasets,make_dir

# 数据加载
data_Dir = "/home/admin123/SATData/data"
# 实例化数据集
emg_data, angle_data = make_datasets(data_Dir, peopleList=['S01'], exp_class="MJ", cluster_num=6, fusionMethod="PCA", windowLength=256, stepLength=1, delta_T=20)

semgData = torch.tensor(emg_data, dtype=torch.float32)
angleData = torch.tensor(angle_data, dtype=torch.float32)
print("semg 数据形状为：", semgData.shape)
print("angle 数据形状为：", angleData.shape)

dataset = TensorDataset(semgData, angleData)

# 定义划分比例
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
# 数据集分割
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# 数据加载
train_loader = DataLoader(train_dataset, batch_size=256, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=256, shuffle=False)

# 初始化模型、优化器和损失函数
model = MyoNet(PreNum=3)
modelName = "MyoNet"
model = model.cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.MSELoss()
criterion = criterion.cuda()

formatted_time = datetime.now().strftime("%m-%d-%H:%M:%S")
checkpoint_save_Dir = os.path.join("/home/admin123/SATData", "Run", modelName, formatted_time)
make_dir(checkpoint_save_Dir)
checkpoint_save_path = os.path.join(checkpoint_save_Dir,"best.pth")

best_loss = float('inf')
train_running_loss_ls = []
test_running_loss_ls = []
test_accuracies = []
for epoch in range(1, 2000+1):
    model.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.cuda(), labels.cuda()
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    train_running_loss_ls.append(running_loss)

    
    # 测试模型
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.cuda(), labels.cuda()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            test_loss += loss
    test_running_loss_ls.append(test_loss)
    # if epoch % 50 == 0:
    print('Epoch %d, Train Loss: %.5f, Test Loss: %.5f'%(epoch, running_loss, test_loss*4))

    if running_loss < best_loss:
        best_loss = running_loss
        torch.save(model.state_dict(), checkpoint_save_path)

semg 数据形状为： torch.Size([26645, 6, 256])
angle 数据形状为： torch.Size([26645, 3])
Epoch 1, Train Loss: 325654.34644, Test Loss: 290318.56250
Epoch 2, Train Loss: 265531.07300, Test Loss: 243791.54688
Epoch 3, Train Loss: 224380.09595, Test Loss: 207002.84375
Epoch 4, Train Loss: 191052.89697, Test Loss: 176799.70312
Epoch 5, Train Loss: 163461.04541, Test Loss: 151842.39062
Epoch 6, Train Loss: 140888.51709, Test Loss: 131101.42188
Epoch 7, Train Loss: 121787.07764, Test Loss: 113879.53125
Epoch 8, Train Loss: 106002.44348, Test Loss: 99544.74219
Epoch 9, Train Loss: 92921.92938, Test Loss: 87609.17969
Epoch 10, Train Loss: 82021.66486, Test Loss: 77666.82812
Epoch 11, Train Loss: 72906.71930, Test Loss: 69379.32812
Epoch 12, Train Loss: 65392.47980, Test Loss: 62497.68359
Epoch 13, Train Loss: 59130.54102, Test Loss: 56791.44141
Epoch 14, Train Loss: 53881.25226, Test Loss: 52072.51172
Epoch 15, Train Loss: 49523.92825, Test Loss: 48191.97656
Epoch 16, Train Loss: 46033.59262, Test Loss: 45

In [2]:
import csv
# 计算性能指标
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

file_path = os.path.join(checkpoint_save_Dir, "output.csv")
# 将两个列表写入到 CSV 文件
with open(file_path, 'w', newline='', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    
    # 写入表头（可选）
    writer.writerow(["train_loss", "test_loss"])
    
    # 写入数据
    for item1, item2 in zip(train_running_loss_ls, test_running_loss_ls):
        writer.writerow([item1, item2])
print(f"loss 文件已保存到: {file_path}")

loss 文件已保存到: /home/admin123/SATData/Run/MyoNet/04-13-14:24:40/output.csv


In [ ]:
train_running_loss_ls

[]